### Import packages

In [1]:
# Add python code here

# Make your own: 0-D Perfectly Stirred Reactor

The perfectly stirred reactor (aka "PSR") is defined by a relatively simple set of governing equations, providing an excellent opportunity to demonstrate how we can use Cantera as a "calculator" to provide the terms and variables required for solving a set of equations of our own choosing.

<img src="PSR.png" alt="Cartoon of a Perfectly Stirred Chemical Reactor." style="width: 500px;"/>

The model includes user inputs for the inlet gas conditions (temperature, pressure, composition) and mass flux (typically defined in terms of the residence time $\tau$), some condition for the outlet flow, and an optional heat transfer boundary condition.

## Differential equations:

Given the above, the state of the reactor is defined entirely by the temperature $T$ and the mass densities of each species $\rho_k$.  Note that we could use some combination of mass/mole fractions and pressure or total mass density.  But $\rho = \sum_k \rho_k$ and $Y_k = \frac{\rho_k}{\rho}$ provides sufficient information to fix the reactor state.

### Species mass densities:
The species mass densities change due to three phenomena:
- Flow rate of species in: $\dot{m}_{\rm in} Y_{k,\,{\rm in}}$
- Flow rate of species out: $\dot{m}_{\rm out} Y_{k}$, where $\dot{m}_{\rm out} = K\left(P - P_{\rm out}\right)$ and $Y_k$ is the current composition of the reactor contents. $K$ is a user-defined "valve" coefficient for the outlet flow.
- Homogeneous chemical reactions occuring within the reactor, which occur at a net production molar rate of $\dot \omega_k\,\left[{\rm{unit}}: \frac{{\rm kmol}_k}{\rm m^3 \cdot s}\right]$.

We will pose all state and process variables with respect to the total reactor volume. Hence, for a gas-phase species $k$, we have:

<div class="alert-danger">
    $$ \frac{d\rho_k}{dt} =  \dot{m}^{\prime\prime\prime}_{\rm in}Y_{k,\,{\rm in}} - Y_kK\left(P - P_{\rm out}\right) + W_k \dot \omega_k$$
</div>
where $W_k$ is the species molecular weight $\left[{\rm{unit:}} \frac{\rm kg}{{\rm kmol}_k}\right]$. The term $\dot{m}^{\prime\prime\prime}_{\rm in}$ is inlet mass flow rate per unit volume of the reactor and is calculated as 
<div class="alert-danger">
    $$ \dot{m}^{\prime\prime\prime}_{\rm in} = \frac{\dot m_{\rm in}}{V} = \frac{\rho}{\tau}, $$
</div>
where $\tau$ is the residence time.


### Temperature:
Assuming constant volume and a single-inlet single-exit reactor, the temperature of an ideal gas reactor is described on the [Cantera web page](https://cantera.org/science/reactors.html#ideal-gas-reactor) as:

<div class="alert-danger">
    $$ \frac{dT}{dt} = \frac{1}{\rho c_v}\left(-q + \dot{m}^{\prime\prime\prime}_{\rm in}\left(h_{\rm in} - \sum_k u_kY_{k,{\rm in}}\right) - \frac{p}{\rho}\dot{m}^{\prime\prime\prime}_{\rm out}-\sum_k\omega_k u_k \right)$$
</div>

where, again, all mass flow rates $\dot{m}^{\prime\prime\prime}$ are per unit volume of reactor and $q$ is an optional heat loss term. $u_k$ are species specific internal energies. The outlet mass flow rate is $\dot{m}^{\prime\prime\prime}_{\rm out} = K (P - P_{\rm out})$.

### Define the residual:

In [2]:
# Add python code here

### User inputs:

In [3]:
# Add python code here

### Instantiate Cantera objects:

In [4]:
# Add python code here

### Initialize the Solution Vector and compute solution:

In [5]:
# Add python code here

### Some basic plotting:

In [6]:
# Add python code here

### Challenge question:

The stirred-reactors are typically operated in steady-state. The current model is also used to evaluate the steady-state behavior of the reactor. Can you extend this model to predict a transient behavior? In particular try looking at the effects of imposing time-varying operating conditions such as

a. Reactor volume, $V(t)$

b. Inlet mass flow rate, $\dot m(t)$

c. Reactor temperature, $T(t)$
